<a href="https://colab.research.google.com/github/Banavath-Vishnu/Bubble-Game/blob/main/AI_PPT_GENERATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  google-generativeai python-pptx pillow requests python-dotenv

In [ ]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
import requests
from PIL import Image
import io
import json
from google.colab import userdata
userdata.get('PEXELS_API_KEY')

load_dotenv()


False

In [ ]:
class PPT_Generator:
  def __init__(self, api_key=None):
    self.api_key = api_key
    if not self.api_key:
      raise ValueError('Error Gemini key')

    genai.configure(api_key=self.api_key)
    self.model = genai.GenerativeModel('gemini-2.5-pro')
    self.model_vision = genai.GenerativeModel('gemini-2.5-pro')

    self.presentation = Presentation()

  def generate_content_outline(self, topic, num_slides=5):
    prompt=f"""
    create a detailed outline for a PowerPoint Presentation on "{topic}" with {num_slides} sides.
    Return the response as a JSON array with the following structure:
    [
      {{
        "title": "Slide Title",
        "content": "Main Content points as bullet points",
        "slide_type":"title\content\image\conclusion"
      }}
    ]

    Make sure the content is engaging, informative and well structured.
    The response must be a valid JSON array.
     """
    try:
     response = self.model.generate_content(prompt)
     content = response.text.strip()

     if "```json" in content:
        content = content.split("```json")[1].split("```")[0].strip()
     elif "```" in content:
        content = content.split("```")[1].strip()

     try:
      return json.loads(content)
     except json.JSONDecodeError as e:
      print(f"Error decoding JSON: {e}")
      return None

    except Exception as e:
      print(f"Error generating content: {e}")
      return None


  def generate_image_description(self, slide_content):
     # Ensure the input is a string before processing
     slide_content_str = str(slide_content)
     prompt = f"""
    Based on this slide content, suggest a relevant image description that would enhance the presentations.
    {slide_content_str}
    return only a brief, descriptive phrase suitable for image search (max 5 words)
    dont return a list object as data
    """

     try:
       response = self.model.generate_content(prompt)
       # Explicitly convert response.text to string before stripping
       content = str(response.text).strip()
       return content
     except Exception as e:
       print(f"Error generating image description: {e}")
       return 'professional presentation'

  def download_image(self, query, save_path = "temp_image.jpg"):
    try:
      url = "https://api.pexels.com/v1/search"
      # Hardcode the Pexels API key as requested
      headers = {
          "Authorization": "<>API KEY<>"
      }
      params = {
          "query": query,
          "per_page": 1,
          "orientation": "landscape"
      }

      response = requests.get(url, headers=headers, params=params)
      response.raise_for_status()

      data = response.json()
      if not data.get('photos'):
        raise ValueError("No photos found")

      image_url = data['photos'][0]['src']['original']
      image_response = requests.get(image_url)
      image_response.raise_for_status()

      with open(save_path, 'wb') as f:
        f.write(image_response.content)

      return save_path
    except Exception as e:
      print(f"Error downloading image: {e}")
      return None

  def create_title_slide(self, title, subtitle=""):
    slide_layout = self.presentation.slide_layouts[0]
    slide = self.presentation.slides.add_slide(slide_layout)

    title_shape = slide.shapes.title
    title_shape.text = title
    title_shape.text_frame.paragraphs[0].font.size = Pt(30)
    title_shape.text_frame.paragraphs[0].font.bold = True
    title_shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
    title_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    if subtitle:
      subtitle_shape = slide.placeholders[1]
      subtitle_shape.text = subtitle
      subtitle_shape.text_frame.paragraphs[0].font.size = Pt(20)
      subtitle_shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
      subtitle_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

  def create_content_slide(self, title, content, include_image):
    slide_layout = self.presentation.slide_layouts[1]
    slide = self.presentation.slides.add_slide(slide_layout)

    title_shape = slide.shapes.title
    title_shape.text = title
    title_shape.text_frame.paragraphs[0].font.size = Pt(20)
    title_shape.text_frame.paragraphs[0].font.bold = True

    # Ensure content is a string before setting it
    if isinstance(content, list):
        content_str = "\n".join(content) # Join list items with newline
    else:
        content_str = str(content) # Convert other types to string

    content_shape = slide.shapes.placeholders[1]
    content_shape.text = content_str

    text_frame = content_shape.text_frame
    for paragraph in text_frame.paragraphs:
      paragraph.font.size = Pt(14)
      paragraph.font.color.rgb = RGBColor(0, 0, 0)

    if include_image:
     try:
      image_desc = self.generate_image_description(content_str) # Pass the string version
      image_path = self.download_image(image_desc)
      if image_path and os.path.exists(image_path):
        slide.shapes.add_picture(image_path, Inches(6), Inches(2), height=Inches(4))
        os.remove(image_path)
     except Exception as e:
      print(f"Error adding image to slide: {e}")

    return slide

  def create_image_slide(self, title, image_query, content=""):
    slide_layout = self.presentation.slide_layouts[8]
    slide = self.presentation.slides.add_slide(slide_layout)

    title_box = slide.shapes.add_textbox(Inches(1), Inches(1), Inches(8), Inches(1))
    title_frame = title_box.text_frame
    title_frame.text = title
    title_frame.paragraphs[0].font.size = Pt(20)
    title_frame.paragraphs[0].font.bold = True
    title_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
    title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    content_box = slide.shapes.add_textbox(Inches(0.5), Inches(2), Inches(4), Inches(5))
    content_frame = content_box.text_frame
    # Ensure content set here is a string
    content_frame.text = str(content) # Initialize with empty text, content will be added if available in outline

    for paragraph in content_frame.paragraphs:
      paragraph.font.size = Pt(20)
      paragraph.font.color.rgb = RGBColor(0, 0, 0)

    try:
      image_path = self.download_image(image_query)
      if image_path and os.path.exists(image_path):
        slide.shapes.add_picture(image_path, Inches(6), Inches(2), height=Inches(4))
        os.remove(image_path)
    except Exception as e:
      print(f"Error adding image to slide: {e}")

    return slide


  def generate_presentation(self, topic, num_slides=5, output_file = "output.pptx"):
    content_outline = self.generate_content_outline(topic, num_slides)

    if not content_outline:
        print("Error: Could not generate content outline.")
        return None

    for i, slide_data in enumerate(content_outline):
      title = slide_data.get("title", f"Slide {i+1}")
      # Ensure content is always a string before any processing
      content = str(slide_data.get("content", ""))
      slide_type = slide_data.get("slide_type", "content")

      print(f"Creating slide {i+1}: {title}")

      if slide_type == "title":
        self.create_title_slide(title, "Created BY BANAVATH VISHNU")
      elif slide_type == "image":
        # Pass the string content to generate_image_description
        img_query = self.generate_image_description(content) if content else title
        # Pass string content to image slide
        self.create_image_slide(title, img_query, content=content)
      elif slide_type == "conclusion":
        # Assuming conclusion slide might have string content as subtitle
        self.create_title_slide(title, subtitle=content)
      else: # Default to content slide
        # Pass the string content to create_content_slide
        self.create_content_slide(title, content, include_image=True)

    self.presentation.save(output_file)
    print(f"Presentation saved to {output_file}")

    return output_file

print('Done')

Done


In [ ]:
api_key = "<>YOUR API KEY<>"

In [ ]:
try:
  generator = PPT_Generator(api_key=api_key)
except Exception as e:
  print(f"Error creating PPT_Generator: {e}")

In [ ]:
topic = "AI IN AGRICULTURE"
num_slides = 5

try:
  output_file = generator.generate_presentation(topic, num_slides)
except Exception as e:
  print(f"Error generating presentation: {e}")

Creating slide 1: AI in Agriculture: Sowing the Seeds of a Smart Future
Creating slide 2: The Global Food Challenge: Why We Need a Revolution
Creating slide 3: Introducing Agri-AI: The Smart Farming Solution
Creating slide 4: Application 1: Precision Farming & Crop Management
Creating slide 5: Application 2: The Rise of the Agri-Bots
Creating slide 6: Application 3: Farming with Foresight
Creating slide 7: Case Study: John Deere's See & Spray™ Technology
Creating slide 8: Hurdles to Adoption: The Road Ahead for Agri-AI
Creating slide 9: The Future Harvest: What's Next?
Creating slide 10: Conclusion: Cultivating a Better Tomorrow
Presentation saved to output.pptx
